# Preprocessing

All four types of inputs need to be pre-processed

1. flow and transaction network data.
2. Shapefile of SA2.
3. Shapefile of road networks.
4. economic indicators, e.g. jobs, employment, etc.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
# import geoplot

## 1. Flow and transaction network data

In [6]:
# mount_path = "/mnt/c/Users/jason/Dropbox (MIT)/"
mount_path = "/Users/shenhaowang/Dropbox (MIT)/project_econ_opportunity_south_Australia"

In [7]:
# read data

### flow
flow_df = pd.read_csv(mount_path + "/SA data/dataSA/flows_sa2_months_2018-02-01.csv")
print("flow shape:", flow_df.shape)
print(flow_df.head(10))
print()

### transaction 1
trans_age_df = pd.read_csv(mount_path + "/SA data/dataSA/transaction_age_bins.csv")
print("trans_age_df: ", trans_age_df.shape)
print(trans_age_df.head(10))
print()

### transaction 2
trans_mcc_df = pd.read_csv(mount_path + "/SA data/dataSA/transaction_mcc.csv")
print("trans_mcc_df: ", trans_mcc_df.shape)
print(trans_mcc_df.head(10))
print()

# Change sa2 values to string
flow_df['sa2'] = flow_df.sa2.astype(str)
flow_df['agent_home_sa2'] = [x[:-2] for x in flow_df['agent_home_sa2'].astype(str)]

trans_age_df['source_sa2'] = trans_age_df['source_sa2'].astype(str)
trans_age_df['target_sa2'] = trans_age_df['target_sa2'].astype(str)

trans_mcc_df['source_sa2'] = trans_mcc_df['source_sa2'].astype(str)
trans_mcc_df['target_sa2'] = trans_mcc_df['target_sa2'].astype(str)

flow shape: (21415, 7)
                       timestamp        sa2  state  agent_home_sa2  \
0  2018-02-01T00:00:00.000+11:00  401011001      4     101021011.0   
1  2018-02-01T00:00:00.000+11:00  404031104      4     101021011.0   
2  2018-02-01T00:00:00.000+11:00  401011001      4     101021012.0   
3  2018-02-01T00:00:00.000+11:00  404031104      4     101021012.0   
4  2018-02-01T00:00:00.000+11:00  406011130      4     101041025.0   
5  2018-02-01T00:00:00.000+11:00  401011001      4     101051539.0   
6  2018-02-01T00:00:00.000+11:00  404031104      4     101051539.0   
7  2018-02-01T00:00:00.000+11:00  404031104      4     101061542.0   
8  2018-02-01T00:00:00.000+11:00  403041073      4     101061544.0   
9  2018-02-01T00:00:00.000+11:00  403041074      4     101061544.0   

   unique_agents  sum_stay_duration  total_stays  
0             80              76375          354  
1            107               6875          175  
2             55             111478          449  
3 

In [8]:
# Shapes
print("flow shape:", flow_df.shape)
print("trans_age_df: ", trans_age_df.shape)
print("trans_mcc_df: ", trans_mcc_df.shape)

flow shape: (21415, 7)
trans_age_df:  (32953, 5)
trans_mcc_df:  (35705, 5)


In [9]:
# describe
print(flow_df.dtypes)

print(trans_age_df.dtypes)

print(trans_mcc_df.dtypes)

timestamp            object
sa2                  object
state                 int64
agent_home_sa2       object
unique_agents         int64
sum_stay_duration     int64
total_stays           int64
dtype: object
source_sa2    object
target_sa2    object
count          int64
amount         int64
age_bin       object
dtype: object
source_sa2    object
target_sa2    object
count          int64
amount         int64
mcc           object
dtype: object


In [10]:
# check a few columns' info
# Note: total number of SA2 in South Australia is 172.
# Many residents' locations from the credit card data are outside South Australia region.
print("--- number of sa2 in flow_df ---")
print("Flow SA2: ", len(np.unique(flow_df.sa2, return_counts = True)[0])) # 
# print(np.unique(flow_df.sa2, return_counts = True)) # 
print("Agent home SA2: ", len(np.unique(flow_df.agent_home_sa2, return_counts = True)[0])) # 
# print(np.unique(flow_df.agent_home_sa2, return_counts = True)) # 

print("--- sa2 in trans_age_df ---")
print("Source sa2: ", len(np.unique(trans_age_df.source_sa2, return_counts = True)[0])) # 
# print(np.unique(trans_age_df.source_sa2, return_counts = True)) # 
print("Target sa2: ", len(np.unique(trans_age_df.target_sa2, return_counts = True)[0])) # 
# print(np.unique(trans_age_df.target_sa2, return_counts = True)) # 

print("--- sa2 in trans_mcc_df ---")
print("Source sa2: ", len(np.unique(trans_mcc_df.source_sa2, return_counts = True)[0])) # 
# print(np.unique(trans_mcc_df.source_sa2, return_counts = True)) # 
print("Target sa2: ", len(np.unique(trans_mcc_df.target_sa2, return_counts = True)[0])) # 
# print(np.unique(trans_mcc_df.target_sa2, return_counts = True)) # 

print("--- activity types in trans_mcc_df ---")
# print(np.unique(trans_mcc_df.mcc, return_counts = True)) # it is activity types.



--- number of sa2 in flow_df ---
Flow SA2:  170
Agent home SA2:  895
--- sa2 in trans_age_df ---
Source sa2:  166
Target sa2:  171
--- sa2 in trans_mcc_df ---
Source sa2:  165
Target sa2:  172
--- activity types in trans_mcc_df ---


In [11]:
# process the three dfs to keep only the agents in SA areas

# flow
south_australia_sa4_set = ['401','402','403','404','405','406','407']
flow_df = flow_df.loc[np.array([x[:3] in south_australia_sa4_set for x in flow_df.agent_home_sa2])]
flow_df = flow_df.loc[np.array([x[:3] in south_australia_sa4_set for x in flow_df.sa2])]

flow_df


,timestamp,sa2,state,agent_home_sa2,unique_agents,sum_stay_duration,total_stays
1716,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217
1717,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322
1718,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334
1719,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487
1720,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278
...,...,...,...,...,...,...,...
20800,2018-02-01T00:00:00.000+11:00,407031166,4,407031170,217,217437,499
20801,2018-02-01T00:00:00.000+11:00,407031167,4,407031170,945,353528,2096
20802,2018-02-01T00:00:00.000+11:00,407031168,4,407031170,580,285362,1599
20803,2018-02-01T00:00:00.000+11:00,407031169,4,407031170,50,3769,93


In [12]:
# flow for adelaide
adelaide_sa4_set = ['401','402','403','404']

flow_adelaide_df = flow_df.loc[np.array([x[:3] in adelaide_sa4_set for x in flow_df.agent_home_sa2])]
flow_adelaide_df = flow_adelaide_df.loc[np.array([x[:3] in adelaide_sa4_set for x in flow_adelaide_df.sa2])]

flow_adelaide_df

,timestamp,sa2,state,agent_home_sa2,unique_agents,sum_stay_duration,total_stays
1716,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217
1717,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322
1718,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334
1719,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487
1720,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278
...,...,...,...,...,...,...,...
15630,2018-02-01T00:00:00.000+11:00,404031105,4,404031109,2304,5876916,12172
15631,2018-02-01T00:00:00.000+11:00,404031106,4,404031109,2967,5413932,16935
15632,2018-02-01T00:00:00.000+11:00,404031107,4,404031109,1811,937583,8366
15633,2018-02-01T00:00:00.000+11:00,404031108,4,404031109,2464,1334692,11329


In [13]:
# transaction age
invalid_value_list = ['Cell Size Limit', 'nan', 'OUTST']
trans_age_df=trans_age_df.loc[~trans_age_df.source_sa2.isin(invalid_value_list)]
trans_age_df=trans_age_df.loc[~trans_age_df.target_sa2.isin(invalid_value_list)]

trans_age_df


,source_sa2,target_sa2,count,amount,age_bin
0,401011001,401011001,192,9903,18-29
1,401011001,401011001,1368,6733,30-39
2,401011001,401011001,1131,4741,40-49
3,401011001,401011001,1145,6021,50-59
4,401011001,401011001,839,3149,60-69
...,...,...,...,...,...
29710,407031170,407031170,804,3891,30-39
29711,407031170,407031170,956,6350,40-49
29712,407031170,407031170,945,6545,50-59
29713,407031170,407031170,738,5061,60-69


In [14]:
# transaction age for adelaide
trans_age_adelaide_df = trans_age_df.loc[np.array([x[:3] in adelaide_sa4_set for x in trans_age_df.source_sa2])]
trans_age_adelaide_df = trans_age_adelaide_df.loc[np.array([x[:3] in adelaide_sa4_set for x in trans_age_adelaide_df.target_sa2])]

trans_age_adelaide_df


,source_sa2,target_sa2,count,amount,age_bin
0,401011001,401011001,192,9903,18-29
1,401011001,401011001,1368,6733,30-39
2,401011001,401011001,1131,4741,40-49
3,401011001,401011001,1145,6021,50-59
4,401011001,401011001,839,3149,60-69
...,...,...,...,...,...
24502,404031109,404031109,99,10260,40-49
24503,404031109,404031109,28,10194,50-59
24504,404031109,404031109,1184,5278,60-69
24505,404031109,404031109,741,2813,70-79


In [15]:
# transaction mcc
# remove the invalid values in source_sa2 and target_sa2
invalid_value_list = ['Cell Size Limit', 'nan', 'OUTST']
trans_mcc_df=trans_mcc_df.loc[~trans_mcc_df.source_sa2.isin(invalid_value_list)]
trans_mcc_df=trans_mcc_df.loc[~trans_mcc_df.target_sa2.isin(invalid_value_list)]

trans_mcc_df


,source_sa2,target_sa2,count,amount,mcc
0,401011001,401011001,4768,6465,Apparel Discount and Department Stores
1,401011001,401011001,1494,11828,Auto and Transportation
2,401011001,401011001,594,7519,Dining Sports and Entertainment
3,401011001,401011001,676,3536,Hardware Supplies and Homewares
4,401011001,401011001,3058,4553,Luxury and Speciality Stores
...,...,...,...,...,...
30927,407031170,407031170,172,9616,Luxury and Speciality Stores
30928,407031170,407031170,1056,14457,Medical and Personal Care
30929,407031170,407031170,5362,14802,Personal Services
30930,407031170,407031170,259,12196,Retail Food Grocery and Supermarkets


In [16]:
# transaction mcc for adelaide
trans_mcc_adelaide_df = trans_mcc_df.loc[np.array([x[:3] in adelaide_sa4_set for x in trans_mcc_df.source_sa2])]
trans_mcc_adelaide_df = trans_mcc_adelaide_df.loc[np.array([x[:3] in adelaide_sa4_set for x in trans_mcc_adelaide_df.target_sa2])]

trans_mcc_adelaide_df


,source_sa2,target_sa2,count,amount,mcc
0,401011001,401011001,4768,6465,Apparel Discount and Department Stores
1,401011001,401011001,1494,11828,Auto and Transportation
2,401011001,401011001,594,7519,Dining Sports and Entertainment
3,401011001,401011001,676,3536,Hardware Supplies and Homewares
4,401011001,401011001,3058,4553,Luxury and Speciality Stores
...,...,...,...,...,...
25600,404031109,404031109,5860,6000,Dining Sports and Entertainment
25601,404031109,404031109,2182,13018,Luxury and Speciality Stores
25602,404031109,404031109,4368,9763,Medical and Personal Care
25603,404031109,404031109,749,3516,Personal Services


#### save cleaned files

In [19]:
trans_mcc_df.to_pickle("../../data/processed_small_data/trans_mcc_df.pkl")

In [20]:
trans_age_df.to_pickle("../../data/processed_small_data/trans_age_df.pkl")

In [21]:
flow_df.to_pickle("../../data/processed_small_data/flow_df.pkl")

In [22]:
flow_adelaide_df.to_pickle("../../data/processed_small_data/flow_adelaide_df.pkl")

In [23]:
trans_age_adelaide_df.to_pickle("../../data/processed_small_data/trans_age_adelaide_df.pkl")

In [24]:
trans_mcc_adelaide_df.to_pickle("../../data/processed_small_data/trans_mcc_adelaide_df.pkl")

## 2. Spatial shapefiles

In [27]:
# Read SA2
sa2_shape = gpd.read_file(mount_path + "/SA data/dataSA/sa2/SA2_2016_AUST.shp")

In [28]:
# Basic info of sa2 for the whole Australia
# sa2 shapefile is quite self-explainary. It includes the names and ids of sa2, sa3, and sa4 areas.
print(type(sa2_shape))
print(sa2_shape.head(10))
print(sa2_shape.dtypes)

<class 'geopandas.geodataframe.GeoDataFrame'>
  SA2_MAIN16 SA2_5DIG16                       SA2_NAME16 SA3_CODE16  \
0  101021007      11007                        Braidwood      10102   
1  101021008      11008                          Karabar      10102   
2  101021009      11009                       Queanbeyan      10102   
3  101021010      11010                Queanbeyan - East      10102   
4  101021011      11011                Queanbeyan Region      10102   
5  101021012      11012  Queanbeyan West - Jerrabomberra      10102   
6  101031013      11013                          Bombala      10103   
7  101031014      11014                            Cooma      10103   
8  101031015      11015                     Cooma Region      10103   
9  101031016      11016            Jindabyne - Berridale      10103   

        SA3_NAME16 SA4_CODE16      SA4_NAME16 GCC_CODE16   GCC_NAME16  \
0       Queanbeyan        101  Capital Region      1RNSW  Rest of NSW   
1       Queanbeyan        

In [29]:
# Keep SA areas.
# Info: file:///Users/shenhaowang/Downloads/StatePublicHealthPlan_Final.pdf (page 32)
south_australia_sa4_set = ['401','402','403','404','405','406','407']
sa2_south_au = sa2_shape.loc[sa2_shape.SA4_CODE16.isin(south_australia_sa4_set)]
print(sa2_south_au.shape) # 172 SA2 regions are left.

(172, 13)


In [30]:
# Keep adelaide areas.
adelaide_sa4_set = ['401','402','403','404']
sa2_adelaide = sa2_shape.loc[sa2_shape.SA4_CODE16.isin(adelaide_sa4_set)]
print(sa2_adelaide.shape)

(110, 13)


In [33]:
sa2_south_au.to_file('../../data/processed_small_data/shapefiles/sa2_south_au.shp')

In [34]:
sa2_adelaide.to_file('../../data/processed_small_data/shapefiles/sa2_adelaide.shp')

# 3. Road networks

In [35]:
# Source: https://data.sa.gov.au/data/dataset/roads/resource/dbd6cc0f-317e-4fc7-b734-b81a518a3f00
sa2_roads = gpd.read_file(mount_path + "/SA data/dataSA/roads/Roads_GDA2020.shp")

In [36]:
print("shape of the road network is: ", sa2_roads.shape)

shape of the road network is:  (457708, 36)


In [37]:
pd.set_option('display.max_columns', None)
sa2_roads.head(5)

,persistent,featurecod,name,roadtype,typesuffix,class,surface,routenum,status,ontype,capturesou,capturemet,featuresou,featurerel,attributer,horizontal,fa_auditda,fa_class,fa_source,fa_method,fa_status,fa_validat,roadusetyp,roaduseaut,road_id,one_way,f_elev,t_elev,crrs_road_,cwy_code,tars_road_,suburbidle,suburbidri,last_edite,shape_Leng,geometry
0,3004811,2013,MULLALONG,TRACK,None,TRK4,UNSE,None,OPER,None,29.0,2.0,1.0,2012-05-17,2012-05-17,10.0,2007-08-17,4.0,1.0,3.0,Validated,2011-03-04,2.0,4.0,31152.0,2W,0.0,0.0,None,U,None,560713.0,560713.0,2016-04-15,0.010374,"LINESTRING (135.19097 -34.46315, 135.19094 -34..."
1,3004805,2013,MORGANS,TRACK,None,TRK4,UNSE,None,OPER,None,29.0,2.0,1.0,2012-05-17,2012-05-17,10.0,2003-03-22,3.0,1.0,3.0,Validated,2011-03-04,2.0,4.0,30679.0,2W,0.0,0.0,None,U,None,560713.0,560713.0,2016-04-15,0.001466,"LINESTRING (135.21053 -34.49383, 135.21057 -34..."
2,3004816,2013,POINT BURGESS,TRACK,None,TRK4,UNSE,None,OPER,None,29.0,2.0,1.0,2012-05-17,2012-05-17,10.0,2006-03-22,3.0,1.0,3.0,Validated,2011-03-04,2.0,4.0,35980.0,2W,0.0,0.0,None,U,None,560713.0,560713.0,2016-04-15,0.000529,"LINESTRING (135.21066 -34.44102, 135.21064 -34..."
3,3288022,2013,None,None,None,TRK2,UNSE,None,OPER,None,26.0,9.0,15.0,2006-01-10,2006-01-10,5.0,None,NaN,NaN,NaN,None,None,NaN,NaN,NaN,2W,0.0,0.0,None,U,None,NaN,NaN,2015-11-18,0.000724,"LINESTRING (131.14305 -26.14934, 131.14318 -26..."
4,123120,2013,None,None,None,TRK4,UNSE,None,OPER,None,29.0,2.0,1.0,2008-03-01,2008-03-01,10.0,2007-08-17,3.0,1.0,3.0,Validated,2011-03-04,2.0,4.0,NaN,2W,0.0,0.0,None,U,None,560713.0,560713.0,2016-04-15,0.002936,"LINESTRING (135.21190 -34.49340, 135.21178 -34..."


In [38]:
# check metadata in the data folder: Roads_GDA2020_metadata.html
for col in sa2_roads.columns:
    print(col)

persistent
featurecod
name
roadtype
typesuffix
class
surface
routenum
status
ontype
capturesou
capturemet
featuresou
featurerel
attributer
horizontal
fa_auditda
fa_class
fa_source
fa_method
fa_status
fa_validat
roadusetyp
roaduseaut
road_id
one_way
f_elev
t_elev
crrs_road_
cwy_code
tars_road_
suburbidle
suburbidri
last_edite
shape_Leng
geometry


In [39]:
sa2_roads.loc[:15,'class']

0     TRK4
1     TRK4
2     TRK4
3     TRK2
4     TRK4
5     TRK4
6     TRK4
7     TRK4
8     LOCL
9     TRK2
10    LOCL
11    LOCL
12    LOCL
13    LOCL
14    LOCL
15    LOCL
Name: class, dtype: object

In [40]:
# focus on only the class variable. 
# Drop 8 null rows. 
sa2_roads = sa2_roads.loc[~sa2_roads['class'].isna(),]


In [41]:
np.unique(sa2_roads['class'], return_counts=True)

(array(['ART', 'BUS', 'COLL', 'FREE', 'HWY', 'LOCL', 'SUBA', 'TRK2',
        'TRK4', 'UND'], dtype=object),
 array([ 15856,    167,  29342,    706,   6439, 187811,  27039, 179227,
         10369,    744]))

In [42]:
# separate three types of roads (class = LOCL, HWY, and UND)
sa2_roads_LOCL = sa2_roads.loc[sa2_roads['class'] == 'LOCL', :]
sa2_roads_HWY = sa2_roads.loc[sa2_roads['class'] == 'HWY', :]
sa2_roads_UND = sa2_roads.loc[sa2_roads['class'] == 'UND', :]

In [43]:
# save
# take a while to save...
sa2_roads.to_file("../../data/processed_small_data/shapefiles/sa2_roads.shp")

In [44]:
# save three categories of roads
sa2_roads_LOCL.to_file("../../data/processed_small_data/shapefiles/sa2_roads_LOCL.shp")
sa2_roads_HWY.to_file("../../data/processed_small_data/shapefiles/sa2_roads_HWY.shp")
sa2_roads_UND.to_file("../../data/processed_small_data/shapefiles/sa2_roads_UND.shp")

In [46]:
# SW Q: How to balance the Github size limits and the need of saving files along the way?


## 4. Jobs and income

In [47]:
jobs_all = pd.read_csv(mount_path + "/SA data/dataSA/SA2_Jobs_All_Jobs_and_Income.csv")
jobs_industries = pd.read_csv(mount_path + "/SA data/dataSA/SA2_Jobs_In_Australia_Employee_Jobs_and_Income.csv")

In [48]:
# process the dataframes' names. Removing the weird space before each column name.

new_idx = []

for col in jobs_industries.columns:
    if col[0]==' ':
        new_idx.append(col[1:])
    else:
        new_idx.append(col)

jobs_industries.columns = new_idx
print(jobs_industries.columns)

new_idx = []

for col in jobs_all.columns:
    if col[0]==' ':
        new_idx.append(col[1:])
    else:
        new_idx.append(col)

jobs_all.columns = new_idx
print(jobs_all.columns)


Index(['cons_med_emply_incjob_aud', 'edu_training_med_emply_incjob_aud',
       'rent_hiring_rlst_med_emply_incjob_aud',
       'arts_rec_med_emply_incjob_aud', 'finance_insur_med_emply_incjob_aud',
       'tot_num_jobs_000', 'tot_med_emply_incjob_aud',
       'utility_med_emply_incjob_aud', 'finance_insur_num_jobs_000',
       'admt_supp_num_jobs_000', 'arts_rec_num_jobs_000',
       'trns_pstl_whse_med_emply_incjob_aud', 'manu_med_emply_incjob_aud',
       'sa2_name16', 'rtl_trade_num_jobs_000',
       'agri_estry_fishing_num_jobs_000', 'hlth_care_soc_asst_num_jobs_000',
       'pro_sci_tech_med_emply_incjob_aud', 'accom_food_med_emply_incjob_aud',
       'rtl_trade_med_emply_incjob_aud', 'pub_admt_safety_num_jobs_000',
       'utility_num_jobs_000', 'trns_pstl_whse_num_jobs_000',
       'whsl_trade_num_jobs_000', 'pro_sci_tech_num_jobs_000',
       'mining_num_jobs_000', 'whsl_trade_med_emply_incjob_aud',
       'accom_food_num_jobs_000', 'admt_supp_med_emply_incjob_aud',
       'pu

In [49]:
# data type of sa2 index
jobs_all['sa2_code16'] = jobs_all['sa2_code16'].astype('str')
jobs_industries['sa2_code16'] = jobs_industries['sa2_code16'].astype('str')

#### save files

In [50]:
jobs_all.to_pickle("../../data/processed_small_data/jobs_all.pkl")

In [51]:
jobs_industries.to_pickle("../../data/processed_small_data/jobs_industries.pkl")